Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

### Инструкция по выполнению проекта

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

### Описание данных

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

# 1. Подготовка

In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import torch
import transformers
import nltk
import re
from nltk.corpus import stopwords as nltk_stopwords
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV
from sklearn.metrics import f1_score, confusion_matrix, precision_score, recall_score
from tqdm import notebook

In [2]:
df = pd.read_csv('/home/konstantin/datasets/toxic_comments.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


In [4]:
df.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


Основная идея нижеследующих блоков в том, чтобы обучить модель без нейронок, посмотреть, сможем ли выжать приемлемый результат без них (хорошо бы, если так), а потом повторить обучение на PyTorch+BERT

In [5]:
train, test = train_test_split(df, random_state=12345)

# 2. Обучение классическим ML

В этом блоке проведем обучение и тест без нейронных сетей

In [6]:
stemmer = SnowballStemmer('english') 

In [7]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/konstantin/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [8]:
def lemmatize(text):
    #lemm_list = stemmer.lemmatize(text)
    lemm_list = [stemmer.stem(word) for word in text]
    lemm_text = "".join(lemm_list)
        
    return lemm_text

def clear_text(text):
    reed = re.sub(r'[^a-zA-Z ]', ' ', text)
    splitted = reed.split()
    joined = " ".join(splitted)
    return joined

Делаем копии выборок, то же сделаем для блока с нейронками, для лучшей сравнимости результатов

In [9]:
train_tweets, test_tweets = train.copy(), test.copy()

Применяем лемматизацию и очистку

In [10]:
train_tweets['lemm_text'] = train['text'].apply(lemmatize)
train_tweets['lemm_text'] = train_tweets['lemm_text'].apply(clear_text)
test_tweets['lemm_text'] = test['text'].apply(lemmatize)
test_tweets['lemm_text'] = test_tweets['lemm_text'].apply(clear_text)

Загружаем список стоп-слов из библиотеки NTLK

In [11]:
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/konstantin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [12]:
corpus_train, corpus_test = train_tweets['lemm_text'].values.astype('U'), \
                            test_tweets['lemm_text'].values.astype('U')

Производим векторизацию (при этом fit делаем только на обучающей выборке)

In [13]:
count_tf_idf = TfidfVectorizer(stop_words=stopwords)
count_tf_idf.fit(corpus_train)
tf_idf_train = count_tf_idf.transform(corpus_train)
tf_idf_test = count_tf_idf.transform(corpus_test)

<h4> Модель логистической регрессии </h4>

In [14]:
model = LogisticRegression(max_iter=300, solver='liblinear', class_weight='balanced')
score1 = cross_val_score(model, tf_idf_train, train.toxic, scoring='f1')

In [15]:
print(np.mean(score1))

0.750752910969583


In [16]:
model.fit(tf_idf_train, train.toxic)

LogisticRegression(class_weight='balanced', max_iter=300, solver='liblinear')

In [17]:
pred_lr = model.predict(tf_idf_test)

In [18]:
 print("F1:", f1_score(test.toxic, pred_lr).round(4), '\nPrecision:',
       precision_score(test.toxic, pred_lr).round(4), '\nRecall', recall_score(test.toxic, pred_lr).round(4))

F1: 0.7588 
Precision: 0.6829 
Recall 0.8537


In [19]:
print(confusion_matrix(test.toxic, pred_lr))

[[34186  1620]
 [  598  3489]]


<h4> Градиентный бустинг на деревьях </h4>

В целях экономии времени, я решил остановиться на одной модели бустинга

In [20]:
model_lgmb = lgb.LGBMClassifier(n_estimators=500)

In [21]:
score2 = cross_val_score(model_lgmb, tf_idf_train, train.toxic, scoring='f1')

In [22]:
print(np.mean(score2).round(4))

0.7741


In [23]:
df.toxic.mean()

0.10167887648758234

# 3. Neural Network

Поскольку обучение нейронок предпочтительно делать на GPU, раздел вынесен в отдельный блокнот (в соседнем файле), либо можно пройти по ссылке непосредственно в Google Colab

https://colab.research.google.com/drive/1XTw1X7BS-_p9h03uH0ffCf_9BTzr7ouS?usp=sharing

# 4. Выводы

В данном проекте мы решали задачу семантической классификации текстов. В ходе работы обучили модели логистической регрессии и градиентного бустинга на деревьях, применив TD-IDF и очистку, используя регулярные выражения. Необходимый уровень целевой метрики F1 был достигнут.

Очень много времени было потрачено на нейросеть BERT (точнее, я использовал вариант Distibert, согласно документации эта модель хорошо подходит именно для задачи классификации. хотя обычный BERT тоже пробовал). К сожалению, в итоге данная модель все равно оказалась сравнима или даже хуже простой логистической регрессии (или бустинга на деревьях), а с учетом времени обучения и затрат вычислительных ресурсов ее попросту нецелесобразно пускать в продакшн. В качестве возможной причины тауой низкой результативности я бы обозначил шум в данных, в частности, я выборочно просмотрел тексты, попадаются и куски иноязычного текста попадаются (не английский), и разного рода сленг. Кроме того, отдельно посмотрел примеры, где модель не смогла правильно выделить целевой класс (ложноотрицательные ответы), если бы я размечал данные, не все бы пометил как токсичные. Т.е. задача неочевидная даже с точки зрения человека, что уж говорить о машине.

Преподаватель также высказывался, что некоторые датасеты с легкостью берет обычная регрессия, а другие даже  лучшим моделям не под силу. У меня впечатление, что мы имеем второй случай. Думаю, модель BERT может выдавать гораздо лучший результат, а если запускать ее на подходящем оборудовании, то и с производительностью не будет больших проблем.